读取分词后的csv文档

In [68]:
import pandas as pd
import numpy as np
from math import log
from collections import Counter

data = pd.read_csv('cut.csv')
test = pd.read_csv('cut_test.csv')
data.head()

,content,estimate
0,外交部 回击 美称 疫情 初期 受 误导 言论 赵立 坚三连 问 回击 美方 言论 今天 针...,0
1,果农 很 烦恼 马来西亚 水果 近一半 被 丢弃 受新冠 肺炎 疫情 影响 马来西亚 农业 ...,2
2,美国 返 沪 的 名 学生 检测 均 为 阴性 关注 新冠 肺炎 月 日 上海市 卫生 健康...,1
3,印度 最大 贫民窟 确诊 病例 破百 累计 死亡 例 当地 时间 日 印度 最大 贫民窟 孟...,2
4,流行病学 调查 在 这次 抗击 新冠 肺炎 战役 中 一个 陌生 字眼 流调 频繁 走进 大...,1


In [69]:
x = data['content']     #存储分词后的内容
y = data['estimate']    #存储用户情感倾向
x_test = test['content']    #存储测试样本的内容
y_test = test['estimate']   #存储测试样本的用户情感倾向

from sklearn.model_selection import train_test_split

# x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=22)

from sklearn.feature_extraction.text import CountVectorizer

def txt2list(wordstxt):
    with open(wordstxt) as f:
        reader = f.read()
    stopwords_list = reader.split('\n')     #按行存储为list
    out_stopwords_list = [i for i in stopwords_list]
    return out_stopwords_list

filename = '停用词表.txt'
stopwords = txt2list(filename)

vect = CountVectorizer(max_df = 0.8, 
                       min_df = 3, 
                       token_pattern=u'(?u)\\b[^\\d\\W]\\w+\\b', 
                       stop_words=frozenset(stopwords))     #词频统计

_list = vect.fit_transform(x).toarray()     #训练样本词频统计
test_list = vect.transform(x_test).toarray()        #检验样本词频统计
a = pd.DataFrame(_list, columns=vect.get_feature_names())       #创建数据帧用于显示
a.head()

,一位,一名,一定,一家,一度,一次,一种,一组,一起,一部,...,面罩,领导小组,风险,首批,首次,马来西亚,高浓度,黑龙江,黑龙江省,鼓励
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,4,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


打印词向量数组

In [82]:
print(_list)
print(test_list)

[[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]]
[[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]]


将结果转化为array

In [83]:
y_array = np.array(y)       #训练样本情感倾向
y_test_array = np.array(y_test)     #检验样本情感倾向
print(y_array)
print(y_test_array)

[0 2 1 2 1 0 1 2 2 0 2 1 1 2 0 0 1 0 2 0 1 0 2 0 2 1 0 1 0 0 2 0 2 2 0 1 0
 1 1 0 2 2 0 0 1 2 1 0 1 1 0 0 0 1 2 2 1 0 0 2 1 2 2 2 0 1 2 2 1 0 2 0 0 0
 2 0 2 1 0 0 1 2 0 0 0 1 2 2 2 0 0 0 0 2 0 2 1 1 0 0 1 2 2 0 0 0 0 1 0 0 0
 2 2 2 1 2 0 1 1 2 2 2 2 1 1 0 2 0 1 0 1 1 2 2 0 0 2 0 1 1 2 0 0 2 2 0 1 2
 1 0 2 0 2 2 2 2 2 0 0 0 2 0 0 0]
[0 2 2 1 1 0 0 2 0 0 2 0 2 2 0 0 2 1 0 0 0 0 2 2 0 2 0 0 2 0 0 0 0 1 0 1 0
 0 1 0 2 1 0 0 0 0 0 0 2 2 0 0 2 1 0 1 0 2 2 0 1 0 0 0 0 0 2 0 0 2 1 1 1 2
 2 1 1 0 2 0 0 2 2 1 0 0 1 2 2 0 0 2 0 1 0 0 1 2 1 0 0 2 2 0 0 2 2 0 2 0 2
 2]


计算各类总数，用于后续计算先验概率

In [99]:
y_static = Counter(y_array.flatten())   #统计情感倾向类型
sumlist = []
sumlist.append(y_static.get(0))
sumlist.append(y_static.get(1))
sumlist.append(y_static.get(2))
sum = sumlist[0]+sumlist[1]+sumlist[2]
print("the sum is : {}\nthe sum of 0 is :{}\nthe sum of 1 is :{}\nthe sum of 2 is :{}".format(sum,sumlist[0],sumlist[1],sumlist[2]))

(164,)
the sum is : 164
the sum of 0 is :67
the sum of 1 is :40
the sum of 2 is :57


计算先验概率

In [85]:
prior = [i/sum for i in sumlist]    #每种倾向发生的次数/总次数
print(prior)

[0.40853658536585363, 0.24390243902439024, 0.3475609756097561]


计算条件概率,m*n个(m为类数,n为特征数量) step1:先将各类的词向量汇总

In [86]:
y_dic = {'0':[],'1':[],'2':[]} #字典初始化

rows,cols = _list.shape#计算训练样本维度

for i in range(rows):
    y_dic[str(y_array[i])].append(_list[i])     #统计每种倾向的词向量
for i in range(3):
    y_dic[str(i)] = np.array(y_dic[str(i)])     #格式转换
print(y_dic)

{'0': array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]], dtype=int64), '1': array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]], dtype=int64), '2': array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 1, 1, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]], dtype=int64)}


step2:各类的词向量做累加

In [103]:
conp = {'0':[],'1':[],'2':[]}
summary_ = []
for i in range(3):
    summary_.append(np.cumsum(y_dic[str(i)],axis=0)[y_dic[str(i)].shape[0]-1])      #计算每个词向量的累加值
print(summary_)

[array([ 1,  1,  0,  1,  0,  1,  0,  1,  0,  2,  2,  0,  0,  4,  2,  0,  3,
        1,  0,  3,  7,  5,  1,  0,  7,  1,  0,  1, 20,  5,  2,  0,  0,  2,
        1,  4,  2,  2,  1,  3,  0,  1,  1,  0,  1,  0, 15,  3,  4,  4,  5,
        3,  0,  3,  1,  0,  0,  4,  0,  4,  4,  0,  4,  2,  0,  1,  1,  9,
        4,  1,  2,  0,  3,  0,  2, 10,  2,  0,  0,  0,  5,  0,  6,  1,  0,
        0,  3,  9,  0,  3,  0,  0,  2,  0,  3,  2,  2,  4,  5,  3,  0,  2,
        2,  0,  4,  1,  1,  2,  2,  8,  6,  5,  2,  3,  4,  2,  1,  4,  4,
        7,  0,  0,  3,  0,  2,  1,  4,  0,  0,  0,  0,  4,  6,  4,  5,  2,
        4,  0,  2,  2,  2,  2,  0,  0,  3,  1,  2,  0,  2,  4,  0,  3,  2,
        1,  3,  0,  5,  8,  1,  2,  2,  0,  2,  0,  2,  3,  0, 18,  2,  1,
        2,  3,  5,  6,  2,  0,  1,  3,  3,  7,  1,  3,  2,  0,  1,  9,  1,
        2,  0,  8,  0,  1,  7,  0,  3,  3,  5,  1, 14,  2,  0,  3,  0,  2,
        7,  3,  1,  1,  0,  1,  2,  7,  6,  7,  2,  0,  2,  0,  8,  3,  1,
        2,  0,  2,  3,  

step3:计算各类中词总数

In [88]:
wordsum = []
for i in range(3):
    wordsum.append(summary_[i].sum(axis = 0))
print(wordsum)

[1321, 734, 1292]


step4:计算条件概率,转为自然对数形式

In [89]:
for i in range(3):
    conp[str(i)] = (summary_[i]+1)/(sumlist[i]+2) #拉普拉斯平滑处理（防止出现0概率对乘除计算的影响）
    for j in range(len(conp[str(i)])):
        conp[str(i)][j] = log(conp[str(i)][j])#转化为对数形式（和条件概率正相关）
print(conp)

 -3.54095932, -4.2341065 , -3.13549422,
       -3.13549422, -4.2341065 , -4.2341065 , -2.62466859, -3.13549422,
       -4.2341065 , -2.84781214, -3.54095932, -4.2341065 , -2.84781214,
       -2.15466496, -2.44234704, -3.54095932, -4.2341065 , -2.15466496,
       -3.54095932, -4.2341065 , -3.54095932, -1.18958407, -2.44234704,
       -3.13549422, -4.2341065 , -4.2341065 , -3.13549422, -3.54095932,
       -2.62466859, -3.13549422, -3.13549422, -3.54095932, -2.84781214,
       -4.2341065 , -3.54095932, -3.54095932, -4.2341065 , -3.54095932,
       -4.2341065 , -1.46151778, -2.84781214, -2.62466859, -2.62466859,
       -2.44234704, -2.84781214, -4.2341065 , -2.84781214, -3.54095932,
       -4.2341065 , -4.2341065 , -2.62466859, -4.2341065 , -2.62466859,
       -2.62466859, -4.2341065 , -2.62466859, -3.13549422, -4.2341065 ,
       -3.54095932, -3.54095932, -1.93152141, -2.62466859, -3.54095932,
       -3.13549422, -4.2341065 , -2.84781214, -4.2341065 , -3.13549422,
       -1.83621123, -3.1

预测训练样本结果并对比计算正确率

In [90]:
y_results = []
for i in range(rows):
    p0 = p1 = p2 = 0
    for j in range(cols):
        p0 = p0 + ((conp['0'][j])*(_list[i][j]))    #条件概率*先验概率
        p1 = p1 + ((conp['1'][j])*(_list[i][j]))
        p2 = p2 + ((conp['2'][j])*(_list[i][j]))
    if(p0 > p1 and p0 > p2):y_results.append(0)
    elif(p1 > p0 and p1 > p2):y_results.append(1)
    elif(p2 > p0 and p2 > p1):y_results.append(2)
    else:y_results.append(0)
score = 1-(np.int64((y_results-y_array) != 0).sum())/(np.int64((y_results-y_array) != 0).shape[0])
print(score)

0.9512195121951219


预测测试样本并计算正确率

In [91]:
y_test_results = []
test_rows, a = test_list.shape #只用到test_rows变量
print(test_rows)
for i in range(test_rows):
    p0 = p1 = p2 = 0
    for j in range(cols):
        p0 = p0 + ((conp['0'][j])*(test_list[i][j]))
        p1 = p1 + ((conp['1'][j])*(test_list[i][j]))
        p2 = p2 + ((conp['2'][j])*(test_list[i][j]))
    if(p0 > p1 and p0 > p2):y_test_results.append(0)
    elif(p1 > p0 and p1 > p2):y_test_results.append(1)
    elif(p2 > p0 and p2 > p1):y_test_results.append(2)
    else:y_test_results.append(0)
test_score = 1-(np.int64((y_test_results-y_test_array) != 0).sum())/(np.int64((y_test_results-y_test_array) != 0).shape[0])
print(test_score)

112
0.6607142857142857


使用sklearn库中朴素贝叶斯算法预测训练样本

In [92]:
from sklearn.naive_bayes import MultinomialNB
nb = MultinomialNB()

x_train_vect = vect.fit_transform(x)
nb.fit(x_train_vect, y)
train_score = nb.score(x_train_vect, y)
print(train_score)

0.9329268292682927


使用sklearn库预测测试样本

In [93]:
x_test_vect = vect.transform(x_test)
print(nb.score(x_test_vect, y_test))

0.6339285714285714
